## Input some data for generating "constant events"

In [ ]:
import pandas as pd

# Read the data
weather_train_csv = '/Users/zhenzhili/MASTERTHESIS/datasets/#ASHRAE-energy-prediction-dataset/weather_train.csv'
df_wt = pd.read_csv(weather_train_csv)
weather_site_0 = df_wt[df_wt["site_id"]==0]

In [ ]:
clusters = {
    "one": {
        "equipments": {
            "server_1": {
                "hot_aisle": ["KMS_1", "KMS_2"],
                "cold_aisle": ["KMS_3", "KMS_4"]
            },
            "server_2": {
                "hot_aisle": ["KMS_5", "KMS_6"],
                "cold_aisle": ["KMS_7", "KMS_8"]
            }
        }
    },
    "two": {
        "equipments": {
            "server_3": {
                "hot_aisle": ["KMS_9", "KMS_10"],
                "cold_aisle": ["KMS_11", "KMS_12"]
            }
        }
    }
}

item_form = """ 
Form:"date","Appliances","lights","T1","RH_1","T2","RH_2","T3","RH_3","T4","RH_4","T5","RH_5","T6","RH_6","T7","RH_7","T8","RH_8","T9","RH_9","T_out","Press_mm_hg","RH_out","Windspeed","Visibility","Tdewpoint","rv1","rv2"
Details:
date, time year-month-day hour:minute:second
Appliances, energy use in Wh
lights, energy use of light fixtures in the house in Wh
T1, Temperature in kitchen area, in Celsius
RH_1, Humidity in kitchen area, in %
T2, Temperature in living room area, in Celsius
RH_2, Humidity in living room area, in %
T3, Temperature in laundry room area
RH_3, Humidity in laundry room area, in %
T4, Temperature in office room, in Celsius
RH_4, Humidity in office room, in %
T5, Temperature in bathroom, in Celsius
RH_5, Humidity in bathroom, in %
T6, Temperature outside the building (north side), in Celsius
RH_6, Humidity outside the building (north side), in %
T7, Temperature in ironing room , in Celsius
RH_7, Humidity in ironing room, in %
T8, Temperature in teenager room 2, in Celsius
RH_8, Humidity in teenager room 2, in %
T9, Temperature in parents room, in Celsius
RH_9, Humidity in parents room, in %
To, Temperature outside (from Chievres weather station), in Celsius
Pressure (from Chievres weather station), in mm Hg
RH_out, Humidity outside (from Chievres weather station), in %
Wind speed (from Chievres weather station), in m/s
Visibility (from Chievres weather station), in km
Tdewpoint (from Chievres weather station), Â°C
rv1, Random variable 1, nondimensional
rv2, Random variable 2, nondimensional
"""

facts = """Currently, we have the following clusters and their respective equipments and sensors, KMS are sensors that integrated with following functions, like temperature, humidity, dew point, air quality.
The relationship between equipements are as follows:
"""
for cluster, data in clusters.items():
    facts += f"Cluster: {cluster}\n"
    for equipment, details in data["equipments"].items():
        facts += f"  Equipment: {equipment}\n"
        facts += f"    Hot Aisle Sensors: {', '.join(details['hot_aisle'])}\n"
        facts += f"    Cold Aisle Sensors: {', '.join(details['cold_aisle'])}\n"
    facts += "\n"
print(facts)


Currently, we have the following clusters and their respective equipments and sensors, KMS are sensors that integrated with following functions, like temperature, humidity, dew point, air quality.
The relationship between equipements are as follows:
Cluster: one
  Equipment: server_1
    Hot Aisle Sensors: KMS_1, KMS_2
    Cold Aisle Sensors: KMS_3, KMS_4
  Equipment: server_2
    Hot Aisle Sensors: KMS_5, KMS_6
    Cold Aisle Sensors: KMS_7, KMS_8

Cluster: two
  Equipment: server_3
    Hot Aisle Sensors: KMS_9, KMS_10
    Cold Aisle Sensors: KMS_11, KMS_12




# parsing prompts:


In [ ]:
# just record the prompt of following models:
prompt = """
Problem Context: A data center monitors various environmental metrics using sensor data The goal is to detect anomalous events that may indicate failures, inefficiencies, or unexpected conditions in the facility.
Task Description: You are given a serie of facts that description. Your task is to leverage a pre-trained Large Language Model (LLM) combined with symbolic reasoning to analyze natural language event descriptions and map them to formalized logic-based anomaly detection rules.

### Facts are always given in following forms, you should not create this part, they include the sensor's measurements in the time window between "Start_timestamp" and "End_timestamp". Each "Sensor_id" can monitor multiple items, whenever you use facts inside a rule, you should follow the following form: 
Item: sensor_air_temperature, sensor_wind_speed
Form:
sensor_data(Sensor, Item, Start_timestamp, End_timestamp, Std_value, Mean_value, Median_value, Max_value, Min_value, Slope)
Example:
sensor_data(sensor_KMS1, temperature, '2023-08-01T12:00:00', '2023-08-01T12:05:00', 0.8, 25.4, 25.3, 26.1, 24.5, 0.1).
sensor_data(sensor_weather, humidity, '2023-08-05T13:00:00', '2023-08-05T13:05:00', 0.82, 0.80, 0.83, 0.85, 0.78, 0.02).
sensor_data(sensor_cpu, cpuload, '2024-02-01T12:30:00', '2024-02-01T12:35:00', 0.45, 0.42, 0.46, 0.50, 0.40, 0.03).

-----
Below are some examples of how you should construct the predicate according to the input of the user:

### Basic rules thats the objects in datacenter should follow, the name could be directly defined according to it's meaning: 
event_<Event_name1>(Sensor1,Sensor2)

Example:
Discribtion: if both Sensor1 and Sensor2 exceed the threshold of the temperature(in this case 26), the Event_name is set to "temperature_exceed", and the predicate structure is setted as follow:
event_temperature_exceed(Sensor1, Sensor2):-
    sensor_data(Sensor1, temperature, _,_, _, Mean1, _, _, _, Slope1),
    sensor_data(Sensor2, temperature, _, _, _, Mean2, _, _, _, Slope2),
    Slope1 > 0, Slope2 > 0, Mean1 > 26, Mean2 > 26.

Discribtion: if the cpu load is higher as 0.45, the Event_name is set to "cpu_high_load", and the predicate structure is setted as follow:
event_cpu_high_load(Sensor) :- 
  sensor_data(Sensor, cpuload, _, _, _, MeanCPU, _, _, _, _), MeanCPU > 0.45.

Discribtion: if the temperature difference between Sensor1 and Sensor2 is higher that 2.0, the Event_name is set to "temperature_difference_large", and the predicate structure is setted as follow:
event_temperature_difference_large(Sensor1, Sensor2) :- 
  sensor_data(Sensor1, temperature, _, _, _, Mean1, _, _, _, _),
  sensor_data(Sensor2, temperature, _, _, _, Mean2, _, _, _, _), 
  Diff is abs(Mean1 - Mean2), Diff > 2.0.

### Eventually, based on the events that we defined, we should define the anormaly predicate for the anormaly detection:
anormaly(<Anormaly_name>) :-

Example:
Description: When the CPU is highly loaded and the temperature rises, the Anormaly_name is set to high_load_phenomenon, and the predicate structure is setted as follow:
anomaly(high_load_phenomenon) :-
    event_cpu_high_load(sensor_cpu),
    event_temperature_rise(sensor_KMS1).

Description: When the CPU is highly loaded and the temperature drops, the Anormaly_name is set to temperature_sensor_failure, and the predicate structure is setted as follow:
anomaly(temperature_sensor_failure) :-
    event_cpu_high_load(sensor_cpu),
    event_temperature_fall(sensor_KMS1).

Description: When the temperature difference between sensor_KMS1 and sensor_KMS2 is too large, the Anormaly_name is set to cooling_system_failure, and the predicate structure is setted as follow:
anomaly(cooling_system_failure) :-
    event_temperature_difference_large(sensor_KMS1, sensor_KMS2).

-----
Below is the one you need to translate:

1. When wind speed rises and temperature drops, there is a risk of catching a cold.
2. When wind speed drops and temperature rises and the temperature is higher than 9, there is a risk of dryness.
3. when wind speed is lower than 3.6, there's a risk of stillness

###
Make sure you capture all the information from the context in the translation.
Make sure you follow the instruction and example to translate the information.
"""

#### so there're some generated models based on above prompt, they are classified by their LLM

In [15]:
model_gpto3mini = """
anomaly(catching_cold) :-
    event_wind_speed_rise(Sensor1),
    event_temperature_drop(Sensor2).

event_wind_speed_rise(Sensor) :-
    sensor_data(Sensor, wind_speed, _, _, _, _, _, MaxWS, _, SlopeWS),
    SlopeWS > 0, MaxWS > 0.

event_temperature_drop(Sensor) :-
    sensor_data(Sensor, temperature, _, _, _, _, _, _, MinTemp, SlopeT),
    SlopeT < 0, MinTemp < 0.

anomaly(dryness_risk) :-
    event_wind_speed_drop(Sensor1),
    event_temperature_rise_higher_9(Sensor2).

event_wind_speed_drop(Sensor) :-
    sensor_data(Sensor, wind_speed, _, _, _, _, _, _, MinWS, SlopeWS),
    SlopeWS < 0, MinWS < 0.

event_temperature_rise_higher_9(Sensor) :-
    sensor_data(Sensor, temperature, _, _, _, MeanT, _, MaxT, _, SlopeT),
    SlopeT > 0, MeanT > 9, MaxT > 9.

anomaly(stillness_risk) :-
    event_low_wind_speed(Sensor).

event_low_wind_speed(Sensor) :-
    sensor_data(Sensor, wind_speed, _, _, _, MeanWS, _, _, _, _),
    MeanWS < 3.6.
"""

model_gpt4o = """ 
% Rule 1: When wind speed rises and temperature drops, there is a risk of catching a cold.
% Define the event predicates:
event_wind_speed_rise(Sensor) :-
    sensor_data(Sensor, sensor_wind_speed, _Start, _End, _Std, _MeanWS, _Median, _Max, _Min, SlopeWS),
    SlopeWS > 0.

event_temperature_fall(Sensor) :-
    sensor_data(Sensor, sensor_air_temperature, _Start, _End, _Std, _MeanT, _Median, _Max, _Min, SlopeT),
    SlopeT < 0.

% Combine events into an anomaly predicate.
anomaly(catching_cold) :-
    event_wind_speed_rise(sensor_WS1),
    event_temperature_fall(sensor_KMS1).

% Rule 2: When wind speed drops and temperature rises and the temperature is higher than 9, there is a risk of dryness.
% Define the event predicates:
event_wind_speed_drop(Sensor) :-
    sensor_data(Sensor, sensor_wind_speed, _Start, _End, _Std, _MeanWS, _Median, _Max, _Min, SlopeWS),
    SlopeWS < 0.

% Here we combine a rising temperature with an extra threshold on the mean temperature.
event_temperature_rise_high(Sensor) :-
    sensor_data(Sensor, sensor_air_temperature, _Start, _End, _Std, MeanT, _Median, _Max, _Min, SlopeT),
    SlopeT > 0,
    MeanT > 9.

% Combine events into an anomaly predicate.
anomaly(dryness) :-
    event_wind_speed_drop(sensor_WS1),
    event_temperature_rise_high(sensor_KMS1).

% Rule 3: When wind speed is lower than 3.6, there's a risk of stillness.
% Define an event predicate checking the mean wind speed value.
event_wind_speed_low(Sensor) :-
    sensor_data(Sensor, sensor_wind_speed, _Start, _End, _Std, MeanWS, _Median, _Max, _Min, _Slope),
    MeanWS < 3.6.

% Combine into an anomaly predicate.
anomaly(stillness) :-
    event_wind_speed_low(sensor_WS1).
"""

model_deepseek = """ 
event_wind_speed_rise(Sensor) :-
sensor_data(Sensor, wind_speed, _, _, _, _, _, _, _, Slope),
Slope > 0.

event_temperature_drop(Sensor) :-
sensor_data(Sensor, temperature, _, _, _, _, _, _, _, Slope),
Slope < 0.

anomaly(risk_of_cold) :-
event_wind_speed_rise(WindSensor),
event_temperature_drop(TempSensor).

event_wind_speed_drop(Sensor) :-
sensor_data(Sensor, wind_speed, _, _, _, _, _, _, _, Slope),
Slope < 0.

event_temperature_rise_high(Sensor) :-
sensor_data(Sensor, temperature, _, _, _, Mean, _, _, _, Slope),
Slope > 0,
Mean > 9.

anomaly(risk_of_dryness) :-
event_wind_speed_drop(WindSensor),
event_temperature_rise_high(TempSensor).

event_wind_speed_low(Sensor) :-
sensor_data(Sensor, wind_speed, _, _, _, Mean, _, _, _, _),
Mean < 3.6.

anomaly(risk_of_stillness) :-
event_wind_speed_low(WindSensor).
"""

===============================================================================================================

# Main Concept-Deepseek based:

===============================================================================================================

In [ ]:
import os# define the arguments for the model:
from src.functions import get_model_files, get_single_model_file_with_path, prolog_reasoning
context_str ="""

"""
rules_str = """
"""
facts_str = """

"""
question_str_1 = """

"""

In [ ]:
# Main function: (create new model files)

from src.symbcot_groq import Groq_Reasoning_Graph_Baseline
args = [
    context_str,
    rules_str,
    facts_str,
    question_str_1,
    "deepseek-r1-distill-llama-70b", # model_name
    "", # stop_words
    "src/generated", # save_path
    500 # max_new_tokens
]
deepseek_problem_reduction = Groq_Reasoning_Graph_Baseline(*args)
deepseek_problem_reduction.reasoning_graph_generation()

Translating...
model_name:  deepseek-r1-distill-llama-70b
Response saved.


In [24]:
model = """ 
0.8::fact(a).
0.6::fact(b).
allfacts :-
    fact(a), fact(b).

    
query(fact(X)).
query(allfacts).
"""
prolog_reasoning(model)

ProbLog Inference Result：
fact(a) = 0.8000
fact(b) = 0.6000
allfacts = 0.4800


## Validation and View Results:

#### 2025-02-07

In [ ]:
facts_1st_risk = """ 
sensor_data(sensor_KMS1, temperature, _, _, _, 26.0, _, _, _, _). 
sensor_data(sensor_cpu, cpuload, _, _, _, 0.75, _, _, _, _).
"""
facts_2nd_risk = """ 
sensor_data(sensor_weather, windspeed, '2023-08-01T12:10:00', '2023-08-01T12:15:00', 0.8, 2, 4.5, 4.0, 3.5, -0.3).
sensor_data(sensor_KMS1, temperature, '2023-08-01T12:10:00', '2023-08-01T12:15:00', 0.8, 10, 9.5, 10.2, 10.8, 0.3).
"""
facts_3rd_risk= """ 
sensor_data(sensor_weather, windspeed, '2023-08-01T12:20:00', '2023-08-01T12:25:00', 0.8, 3, 3.8, 3.5, 3.2, -0.1).
"""
query = """ 
query(anomaly(_)).
"""
facts ={"facts_cold_risk":facts_1st_risk, 
        "facts_dryness_risk":facts_2nd_risk, 
        "facts_stillness_risk":facts_3rd_risk
}
# read all model files that start with "prolog_model_deepseek_":
model_files = get_model_files("2025-02-07", "deepseek")
for model_key in model_files:
    print(f"===================={model_key}====================")
    for fact_key in facts:
        print(f"------------{fact_key}------------")
        whole_model = facts[fact_key] + model_files[model_key] + query
        result_single = prolog_reasoning(whole_model)


Found 10
====================generated/models/2025-02-07/prolog_model_deepseek_2025-02-08 19:39:38.545957.txt====================
------------facts_cold_risk------------
ProbLog Inference Result：
anomaly(X2) = 0.0000
------------facts_dryness_risk------------
ProbLog Inference Result：
anomaly(dryness_risk) = 1.0000
anomaly(stillness_risk) = 1.0000
------------facts_stillness_risk------------
ProbLog Inference Result：
anomaly(stillness_risk) = 1.0000
====================generated/models/2025-02-07/prolog_model_deepseek_2025-02-08 19:37:40.256262.txt====================
------------facts_cold_risk------------
ProbLog Inference Result：
anomaly(X2) = 0.0000
------------facts_dryness_risk------------
ProbLog Inference Result：
anomaly(X2) = 0.0000
------------facts_stillness_risk------------
ProbLog Inference Result：
anomaly(X2) = 0.0000
====================generated/models/2025-02-07/prolog_model_deepseek_2025-02-08 19:39:43.275474.txt====================
------------facts_cold_risk--------

#### 2025-02-09

In [ ]:
facts_1st_risk = """ 
sensor_data(sensor_KMS1, temperature, '2023-08-15T14:00:00', '2023-08-15T14:05:00', 0.7, 27.5, 1.2, 27.3, 27.6, 0.1).
sensor_data(sensor_cpu, cpuload, '2023-08-15T14:00:00', '2023-08-15T14:05:00', 0.9, 0.82, 0.05, 0.81, 0.83, 0.01).
"""
facts_2nd_risk = """ 
sensor_data(sensor_KMS1, temperature, '2023-08-15T03:00:00', '2023-08-15T03:05:00', 0.6, 23.2, 0.8, 23.0, 23.5, -0.3).
sensor_data(sensor_weather, humidity, '2023-08-15T03:00:00', '2023-08-15T03:05:00', 0.7, 0.88, 0.02, 0.87, 0.89, 0.01).
"""
facts_3rd_risk= """ 
sensor_data(sensor_weather, temperature, '2023-08-15T12:00:00', '2023-08-15T12:05:00', 0.8, 25.5, 1.0, 25.3, 25.8, 0.2).
sensor_data(sensor_KMS1, temperature, '2023-08-15T12:00:00', '2023-08-15T12:05:00', 0.9, 22.8, 1.5, 22.5, 23.0, -0.5).
"""
query = """ 
query(anomaly(_)).
"""
network_define = """ 

"""

facts ={"facts_overheat_risk":facts_1st_risk, 
        "facts_condensation_risk":facts_2nd_risk, 
        "facts_high_power_consumption_risk":facts_3rd_risk
}
# read all model files that start with "prolog_model_deepseek_":
model_files = get_model_files("2025-02-09", "deepseek")
for model_key in model_files:
    print(f"===================={model_key}====================")
    for fact_key in facts:
        print(f"------------{fact_key}------------")
        whole_model = facts[fact_key] + model_files[model_key] + query
        result_single = prolog_reasoning(whole_model)


Found 3
====================generated/models/2025-02-09/prolog_model_deepseek_22:29:35.txt====================
------------facts_overheat_risk------------
ProbLog Inference Result：
anomaly(risk_of_overheating) = 1.0000
------------facts_condensation_risk------------
ProbLog Inference Result：
anomaly(risk_of_condensation) = 1.0000
------------facts_high_power_consumption_risk------------
ProbLog Inference Result：
anomaly(risk_of_high_power_consumption) = 1.0000
====================generated/models/2025-02-09/prolog_model_deepseek_22-28-45.txt====================
------------facts_overheat_risk------------
ProbLog Inference Result：
anomaly(overheat_risk) = 1.0000
------------facts_condensation_risk------------
ProbLog Inference Result：
anomaly(condensation_risk) = 1.0000
------------facts_high_power_consumption_risk------------
ProbLog Inference Result：
anomaly(high_power_consumption) = 1.0000
====================generated/models/2025-02-09/prolog_model_deepseek_22-27-10.txt=============

#### 2025-02-11

In [32]:
facts_1st_risk = """ 
sensor_data(sensor_KMS1, temperature, '2023-08-15T14:00:00', '2023-08-15T14:05:00', 0.7, 27.5, 1.2, 27.3, 27.6, 0.1).
sensor_data(sensor_cpu, cpuload, '2023-08-15T14:00:00', '2023-08-15T14:05:00', 0.9, 0.82, 0.05, 0.81, 0.83, 0.01).
"""
facts_2nd_risk = """ 
sensor_data(sensor_KMS1, temperature, '2023-08-15T03:00:00', '2023-08-15T03:05:00', 0.6, 23.2, 0.8, 23.0, 23.5, -0.3).
sensor_data(sensor_weather, humidity, '2023-08-15T03:00:00', '2023-08-15T03:05:00', 0.7, 0.88, 0.02, 0.87, 0.89, 0.01).
"""
facts_3rd_risk= """ 
sensor_data(sensor_weather, temperature, '2023-08-15T12:00:00', '2023-08-15T12:05:00', 0.8, 25.5, 1.0, 25.3, 25.8, 0.2).
sensor_data(sensor_KMS1, temperature, '2023-08-15T12:00:00', '2023-08-15T12:05:00', 0.9, 22.8, 1.5, 22.5, 23.0, -0.5).
"""
query = """ 
query(anomaly(_)).
"""
network_define = """ 

"""

facts ={"facts_overheat_risk":facts_1st_risk, 
        "facts_condensation_risk":facts_2nd_risk, 
        "facts_high_power_consumption_risk":facts_3rd_risk
}
# read all model files that start with "prolog_model_deepseek_":
model_files = get_model_files("2025-02-11", "deepseek")
for model_key in model_files:
    print(f"===================={model_key}====================")
    for fact_key in facts:
        print(f"------------{fact_key}------------")
        whole_model = facts[fact_key] + model_files[model_key] + query
        result_single = prolog_reasoning(whole_model)


Found 0


# real Test:

In [33]:
from functions import get_model_files, get_single_model_file_with_path, prolog_reasoning
model_network_file = "/Users/zhenzhili/MASTERTHESIS/#Expert_System_Design/generated/example_for_netowork/prolog_model_deepseek_2025-02-08.pl"

facts_1st_risk = """ 
sensor_data(sensor_KMS1, temperature, '2023-08-15T14:00:00', '2023-08-15T14:05:00', 0.7, 27.5, 1.2, 27.3, 27.6, 0.1).
sensor_data(sensor_cpu, cpuload, '2023-08-15T14:00:00', '2023-08-15T14:05:00', 0.9, 0.82, 0.05, 0.81, 0.83, 0.01).
"""

facts_2nd_risk = """ 
sensor_data(sensor_KMS1, temperature, '2023-08-15T03:00:00', '2023-08-15T03:05:00', 0.6, 23.2, 0.8, 23.0, 23.5, -0.3).
sensor_data(sensor_weather, humidity, '2023-08-15T03:00:00', '2023-08-15T03:05:00', 0.7, 0.88, 0.02, 0.87, 0.89, 0.01).
"""

facts_3rd_risk= """ 
sensor_data(sensor_weather, temperature, '2023-08-15T12:00:00', '2023-08-15T12:05:00', 0.8, 25.5, 1.0, 25.3, 25.8, 0.2).
sensor_data(sensor_KMS1, temperature, '2023-08-15T12:00:00', '2023-08-15T12:05:00', 0.9, 22.8, 1.5, 22.5, 23.0, -0.5).
"""

query = """ 
query(anomaly(_)).
"""

network_define = """ 
"""

facts ={"facts_overheat_risk":facts_1st_risk, 
        "facts_condensation_risk":facts_2nd_risk, 
        "facts_high_power_consumption_risk":facts_3rd_risk
}
# read all model files that start with "prolog_model_deepseek_":
model_files = get_single_model_file_with_path(model_network_file)
for fact_key in facts:
    print(f"------------{fact_key}------------")
    whole_model = facts[fact_key] + model_files + query
    result_single = prolog_reasoning(whole_model)


this file could not be found


UnboundLocalError: local variable 'model_string' referenced before assignment

### write for csv to prolog, data flow

In [ ]:
import pandas as pd
import sys

def csv_to_prolog(csv_file, output_file):
    # 读取 CSV 文件
    df = pd.read_csv(csv_file)
    
    # 确保 CSV 包含所有必要的列
    required_columns = ["Sensor", "Item", "Start_timestamp", "End_timestamp", "Std_value", "Mean_value", "Median_value", "Max_value", "Min_value", "Slope"]
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")
    
    # 生成 Prolog 规则
    with open(output_file, 'w') as f:
        for _, row in df.iterrows():
            prolog_fact = f"sensor_data({row['Sensor']}, {row['Item']}, '{row['Start_timestamp']}', '{row['End_timestamp']}', {row['Std_value']}, {row['Mean_value']}, {row['Median_value']}, {row['Max_value']}, {row['Min_value']}, {row['Slope']})."
            f.write(prolog_fact + '\n')
    
    print(f"Prolog facts saved to {output_file}")

if len(sys.argv) != 3:
    print("Usage: python csv_to_prolog.py <input_csv> <output_pl>")
    sys.exit(1)

csv_file = sys.argv[1]
output_file = sys.argv[2]

csv_to_prolog(csv_file, output_file)


In [3]:
from functions import get_model_files, get_single_model_file_with_path, prolog_reasoning

model = """ 
sensor_data(Sensor, temperature, _, _, _, 29, _, _, _, _).

temp_exceed_net(Mean, P):-
    Mean > 28, P is 0.8.
    
temp_exceed_net(Mean, P):-
    Mean > 24, Mean =< 28, P is 0.5.


P::event_temperature_exceed(Sensor) :- 
    sensor_data(Sensor, temperature, _, _, _, Mean, _, _, _, _),
    Mean > 26,  % 保留条件（可选）
    temp_exceed_net(Mean, P).  % 神经网络预测概率
    
query(event_temperature_exceed(_)).
"""

prolog_reasoning(model)

NonGroundProbabilisticClause: Encountered a non-ground probabilistic clause at 11:4.

In [7]:
import pandas as pd
Data_Center_Anomalies_with_Nois_path = "/Users/zhenzhili/MASTERTHESIS/datasets/Data_Center_Anomalies_with_Noise.csv"
df_anormales = pd.read_csv(Data_Center_Anomalies_with_Nois_path)

# 创建单独的 CSV 文件
csv_temp = "/mnt/data/temperature_data.csv"
csv_cpu = "/mnt/data/cpu_load_data.csv"
csv_humidity = "/mnt/data/humidity_data.csv"

# 创建单独的 DataFrame
df_temp = df_anormales[['Start_timestamp', 'End_timestamp', 'Temperature']]
df_cpu = df_anormales[['Start_timestamp', 'End_timestamp', 'CPU_load']]
df_humidity = df_anormales[['Start_timestamp', 'End_timestamp', 'Humidity']]

# 保存文件
df_temp.to_csv(csv_temp, index=False)
df_cpu.to_csv(csv_cpu, index=False)
df_humidity.to_csv(csv_humidity, index=False)



OSError: Cannot save file into a non-existent directory: '/mnt/data'

In [ ]:
model = """
t(0.5) :: col(1,red); t(0.5) :: col(1,blue).
t(0.333) :: col(2,red); t(0.333) :: col(2,green); t(0.333) :: col(2,blue).
t(0.5) :: coin_heads.
